<a href="https://colab.research.google.com/github/lrssv/TimeSeriesForecastingWindSun/blob/master/AUTOPARAMETERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUTOARIMA e AUTOSARIMA para o projeto WindSun 

## Obtendo os dados atráves do github

In [ ]:
!pip install pmdarima

In [ ]:
#imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
import statsmodels.api as sm
#from pmdarima.arima import ndiffs
#import pmdarima as pm
import time 

In [ ]:
#get data
url1 = 'https://raw.githubusercontent.com/lrssv/TimeSeriesForecastingWindSun/master/series_ventovel_pampulha_2018_2019'
url2 = 'https://raw.githubusercontent.com/lrssv/TimeSeriesForecastingWindSun/master/series_radiacao_pampulha_2018_2019'

df_ventovel = pd.read_csv(url1, header=0, parse_dates=[0], index_col=0, squeeze=True)
df_radiacao = pd.read_csv(url2, header=0, parse_dates=[0], index_col=0, squeeze=True)


series_ventovel = pd.Series(df_ventovel)
series_radiacao = pd.Series(df_radiacao)

## AUTOARIMA

In [ ]:
#cold start = 1 ano
size = int(len(series_radiacao) / 2)
cold_start = len(series_radiacao[:size])

###Definindo o valor de d para cada série

In [ ]:
def d_term(series,cold_start):

  begin = series.index.min() + timedelta(days=0)
  date_list = [begin + timedelta(hours=cold_start) + timedelta(hours=x) for x in range(0, (len(series)-cold_start), 745)]

  for date in date_list:  
    train = series[begin:date]
    kpss_diffs = ndiffs(train, alpha=0.05, test='kpss', max_d=6)
    adf_diffs = ndiffs(train, alpha=0.05, test='adf', max_d=6)
    n_diffs = max(adf_diffs, kpss_diffs)
    print(n_diffs)

In [ ]:
d_ventovel = d_term(series_ventovel,cold_start)

In [ ]:
df_radiacao = d_term(series_radiacao,cold_start)

###Implementação do AUTOARIMA

In [ ]:
def run_autoarima(series,d,cold_start):
  results = []

  begin = series.index.min() + timedelta(days=0)
  date_list = [begin + timedelta(hours=cold_start) + timedelta(hours=x) for x in range(0, (len(series)-cold_start), 745)]
  
  start = time.time()
  
  for date in tqdm(date_list):  
    train = series[begin:date]
    auto = pm.auto_arima(train, d=d, seasonal=False, stepwise=True,
                     suppress_warnings=True, error_action="ignore", max_p=12, 
                     max_q=12, max_order=None, trace=True)
    results.append(auto.order)

  t = time.time() - start
  return results,t

### Autoparamêtros para a variável: Velocidade do vento

In [ ]:
autoarima_ven = run_autoarima(series_ventovel,1,cold_start)

In [ ]:
autoarima_ven

([(0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0),
  (0, 0, 0)],
 7.342899799346924)

### Autoparamêtros para a variável: Radiação

In [ ]:
autoarima_rad = run_autoarima(series_radiacao,0,cold_start)

In [ ]:
autoarima_rad

([(4, 0, 2),
  (3, 0, 2),
  (4, 0, 6),
  (4, 0, 2),
  (4, 0, 2),
  (4, 0, 6),
  (3, 0, 2),
  (3, 0, 2),
  (4, 0, 2),
  (5, 0, 7),
  (4, 0, 2),
  (6, 0, 7)],
 5395.889846801758)

## AUTOSARIMA

In [ ]:
def run_autosarima(series,d,cold_start):
  results = []

  begin = series.index.min() + timedelta(days=0)
  date_list = [begin + timedelta(hours=cold_start) + timedelta(hours=x) for x in range(0, (len(series)-cold_start), 744)]
  
  start = time.time()

  for date in tqdm(date_list):  
    train = series[begin:date]
    auto = pm.auto_arima(train, d=d, m=4, seasonal=True, stepwise=True,
                     suppress_warnings=True, error_action="ignore", max_p=3, 
                     max_q=3, max_P=3, max_Q=3, trace=True)
    results.append([auto.order,auto.seasonal_order])

  t = time.time() - start

  return results, t

### Autoparamêtros para a variável: Velocidade do vento

In [ ]:
autosarima_ven = run_autosarima(series_ventovel,1,cold_start)

  0%|          | 0/12 [00:00<?, ?it/s]

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[4] intercept   : AIC=inf, Time=23.47 sec
 ARIMA(0,1,0)(0,0,0)[4] intercept   : AIC=112779.454, Time=0.19 sec
 ARIMA(1,1,0)(1,0,0)[4] intercept   : AIC=110263.816, Time=1.22 sec
 ARIMA(0,1,1)(0,0,1)[4] intercept   : AIC=inf, Time=16.82 sec
 ARIMA(0,1,0)(0,0,0)[4]             : AIC=112777.454, Time=0.11 sec
 ARIMA(1,1,0)(0,0,0)[4] intercept   : AIC=110261.816, Time=0.41 sec
 ARIMA(1,1,0)(0,0,1)[4] intercept   : AIC=110263.816, Time=1.24 sec
 ARIMA(1,1,0)(1,0,1)[4] intercept   : AIC=110265.816, Time=1.43 sec
 ARIMA(2,1,0)(0,0,0)[4] intercept   : AIC=109232.083, Time=0.86 sec
 ARIMA(2,1,0)(1,0,0)[4] intercept   : AIC=109234.083, Time=1.69 sec
 ARIMA(2,1,0)(0,0,1)[4] intercept   : AIC=109234.083, Time=1.43 sec
 ARIMA(2,1,0)(1,0,1)[4] intercept   : AIC=109236.083, Time=1.93 sec
 ARIMA(3,1,0)(0,0,0)[4] intercept   : AIC=108668.095, Time=1.08 sec
 ARIMA(3,1,0)(1,0,0)[4] intercept   : AIC=107908.349, Time=8.97 sec
 ARIMA(3,1,0)(2,0

  8%|▊         | 1/12 [23:22<4:17:12, 1402.94s/it]

 ARIMA(2,1,1)(6,0,0)[4]             : AIC=inf, Time=60.00 sec

Best model:  ARIMA(3,1,0)(6,0,0)[4]          
Total fit time: 1402.514 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[4] intercept   : AIC=inf, Time=23.21 sec
 ARIMA(0,1,0)(0,0,0)[4] intercept   : AIC=121582.927, Time=0.19 sec
 ARIMA(1,1,0)(1,0,0)[4] intercept   : AIC=118853.274, Time=1.25 sec
 ARIMA(0,1,1)(0,0,1)[4] intercept   : AIC=inf, Time=8.39 sec
 ARIMA(0,1,0)(0,0,0)[4]             : AIC=121580.927, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[4] intercept   : AIC=118851.275, Time=0.43 sec
 ARIMA(1,1,0)(0,0,1)[4] intercept   : AIC=118853.274, Time=1.25 sec
 ARIMA(1,1,0)(1,0,1)[4] intercept   : AIC=118855.274, Time=1.48 sec
 ARIMA(2,1,0)(0,0,0)[4] intercept   : AIC=117733.899, Time=0.92 sec
 ARIMA(2,1,0)(1,0,0)[4] intercept   : AIC=117735.899, Time=1.74 sec
 ARIMA(2,1,0)(0,0,1)[4] intercept   : AIC=117735.899, Time=1.51 sec
 ARIMA(2,1,0)(1,0,1)[4] intercept   : AIC=117737.899, Time=2.03 sec
 ARIMA(3,1,0

In [ ]:
autosarima_ven

[[(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)],
 [(0, 0, 0), (0, 0, 0, 4)]]

### Autoparamêtros para a variável: Radiação

In [ ]:
autosarima_rad = run_autosarima(series_radiacao,0,cold_start)

  0%|          | 0/12 [00:00<?, ?it/s]

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[4] intercept   : AIC=inf, Time=28.90 sec
 ARIMA(0,0,0)(0,0,0)[4] intercept   : AIC=146539.592, Time=0.16 sec
 ARIMA(1,0,0)(1,0,0)[4] intercept   : AIC=131383.120, Time=2.40 sec
 ARIMA(0,0,1)(0,0,1)[4] intercept   : AIC=136193.882, Time=10.21 sec
 ARIMA(0,0,0)(0,0,0)[4]             : AIC=150115.622, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[4] intercept   : AIC=131388.551, Time=0.43 sec
 ARIMA(1,0,0)(2,0,0)[4] intercept   : AIC=130825.200, Time=17.32 sec
 ARIMA(1,0,0)(3,0,0)[4] intercept   : AIC=130783.114, Time=34.67 sec
 ARIMA(1,0,0)(3,0,1)[4] intercept   : AIC=130556.261, Time=34.37 sec
 ARIMA(1,0,0)(2,0,1)[4] intercept   : AIC=130564.041, Time=18.89 sec
 ARIMA(1,0,0)(3,0,2)[4] intercept   : AIC=130531.513, Time=34.96 sec
 ARIMA(1,0,0)(2,0,2)[4] intercept   : AIC=130542.226, Time=19.73 sec
 ARIMA(1,0,0)(3,0,3)[4] intercept   : AIC=inf, Time=54.63 sec
 ARIMA(1,0,0)(2,0,3)[4] intercept   : AIC=inf, Time=58.52 sec
 ARIMA(0,0,0)(3,

  8%|▊         | 1/12 [21:12<3:53:14, 1272.24s/it]

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[4] intercept   : AIC=inf, Time=28.60 sec
 ARIMA(0,0,0)(0,0,0)[4] intercept   : AIC=159503.734, Time=0.18 sec
 ARIMA(1,0,0)(1,0,0)[4] intercept   : AIC=142884.979, Time=2.57 sec
 ARIMA(0,0,1)(0,0,1)[4] intercept   : AIC=148087.361, Time=11.30 sec
 ARIMA(0,0,0)(0,0,0)[4]             : AIC=163406.754, Time=0.11 sec
 ARIMA(1,0,0)(0,0,0)[4] intercept   : AIC=142887.982, Time=0.47 sec
 ARIMA(1,0,0)(2,0,0)[4] intercept   : AIC=142263.607, Time=12.72 sec
 ARIMA(1,0,0)(3,0,0)[4] intercept   : AIC=142208.475, Time=38.16 sec
 ARIMA(1,0,0)(3,0,1)[4] intercept   : AIC=141954.321, Time=42.23 sec
 ARIMA(1,0,0)(2,0,1)[4] intercept   : AIC=141962.618, Time=24.26 sec
 ARIMA(1,0,0)(3,0,2)[4] intercept   : AIC=141921.785, Time=43.80 sec
 ARIMA(1,0,0)(2,0,2)[4] intercept   : AIC=141934.691, Time=20.79 sec
 ARIMA(1,0,0)(3,0,3)[4] intercept   : AIC=inf, Time=71.33 sec
 ARIMA(1,0,0)(2,0,3)[4] intercept   : AIC=inf, Time=63.57 sec
 ARIMA(0,0,0)(3,

 17%|█▋        | 2/12 [40:01<3:24:54, 1229.42s/it]

...

 83%|████████▎ | 10/12 [5:06:24<1:15:59, 2279.94s/it]

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[4] intercept   : AIC=inf, Time=47.42 sec
 ARIMA(0,0,0)(0,0,0)[4] intercept   : AIC=271573.488, Time=0.45 sec
 ARIMA(1,0,0)(1,0,0)[4] intercept   : AIC=243010.556, Time=4.14 sec
 ARIMA(0,0,1)(0,0,1)[4] intercept   : AIC=251855.469, Time=14.20 sec
 ARIMA(0,0,0)(0,0,0)[4]             : AIC=278242.807, Time=0.19 sec
 ARIMA(1,0,0)(0,0,0)[4] intercept   : AIC=243014.408, Time=1.20 sec
 ARIMA(1,0,0)(2,0,0)[4] intercept   : AIC=241835.771, Time=22.55 sec
 ARIMA(1,0,0)(3,0,0)[4] intercept   : AIC=241785.281, Time=36.10 sec
 ARIMA(1,0,0)(3,0,1)[4] intercept   : AIC=241808.745, Time=71.94 sec
 ARIMA(1,0,0)(2,0,1)[4] intercept   : AIC=241875.713, Time=50.35 sec
 ARIMA(0,0,0)(3,0,0)[4] intercept   : AIC=263796.630, Time=26.12 sec
 ARIMA(2,0,0)(3,0,0)[4] intercept   : AIC=238525.178, Time=72.12 sec
 ARIMA(2,0,0)(2,0,0)[4] intercept   : AIC=238575.739, Time=15.66 sec
 ARIMA(2,0,0)(3,0,1)[4] intercept   : AIC=238460.061, Time=52.74 sec
 A

 92%|█████████▏| 11/12 [5:39:29<36:31, 2191.41s/it]  

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[4] intercept   : AIC=inf, Time=49.61 sec
 ARIMA(0,0,0)(0,0,0)[4] intercept   : AIC=284041.520, Time=0.47 sec
 ARIMA(1,0,0)(1,0,0)[4] intercept   : AIC=254288.881, Time=4.16 sec
 ARIMA(0,0,1)(0,0,1)[4] intercept   : AIC=263508.318, Time=12.55 sec
 ARIMA(0,0,0)(0,0,0)[4]             : AIC=291014.115, Time=0.20 sec
 ARIMA(1,0,0)(0,0,0)[4] intercept   : AIC=254292.900, Time=1.27 sec
 ARIMA(1,0,0)(2,0,0)[4] intercept   : AIC=253086.030, Time=22.79 sec
 ARIMA(1,0,0)(3,0,0)[4] intercept   : AIC=253002.057, Time=52.21 sec
 ARIMA(1,0,0)(3,0,1)[4] intercept   : AIC=252519.020, Time=68.49 sec
 ARIMA(1,0,0)(2,0,1)[4] intercept   : AIC=253105.298, Time=46.10 sec
 ARIMA(1,0,0)(3,0,2)[4] intercept   : AIC=252423.463, Time=62.96 sec
 ARIMA(1,0,0)(2,0,2)[4] intercept   : AIC=252441.963, Time=32.24 sec
 ARIMA(1,0,0)(3,0,3)[4] intercept   : AIC=inf, Time=89.34 sec
 ARIMA(1,0,0)(2,0,3)[4] intercept   : AIC=inf, Time=88.47 sec
 ARIMA(0,0,0)(3,

100%|██████████| 12/12 [6:15:03<00:00, 1875.31s/it]


In [ ]:
autosarima_rad

([[(3, 0, 1), (0, 0, 3, 4)],
  [(1, 0, 1), (3, 0, 2, 4)],
  [(1, 0, 3), (1, 0, 3, 4)],
  [(1, 0, 0), (3, 0, 2, 4)],
  [(2, 0, 1), (0, 0, 3, 4)],
  [(3, 0, 1), (0, 0, 3, 4)],
  [(3, 0, 1), (0, 0, 3, 4)],
  [(3, 0, 1), (0, 0, 3, 4)],
  [(3, 0, 1), (0, 0, 3, 4)],
  [(1, 0, 2), (2, 0, 2, 4)],
  [(2, 0, 1), (3, 0, 0, 4)],
  [(3, 0, 1), (0, 0, 3, 4)]],
 22503.690353393555)